In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Импорт данных через google disk:

In [99]:
import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/Inga/Data/data_project_1.csv')

data.head()

,Unnamed: 0,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


### Обратить внимание на большие значения в столбце days_empl, total_income.

Убираем отрицательный стаж работы с помощью модуля:

In [100]:
data['days_employed'] = data['days_employed'].abs()

### В чем измеряется income?

In [101]:
data.isna().sum()

Unnamed: 0             0
children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

# Обработка пропущенных значений.

Видим, что в столбцах days_emloyed и total_income имеются значения NaN. Заменим их на медианные значения по категории income_type, которая содержит информацию о "типе" занятости клиента.

### Создадим вспомогательный датасет только со столбцами ['income_type', 'total_income', 'days_employed']:

In [114]:
df1 = data[['income_type', 'total_income', 'days_employed']]

In [115]:
df1.head()

,income_type,total_income,days_employed
0,сотрудник,253875.639453,8437.673028
1,сотрудник,112080.014102,4024.803754
2,сотрудник,145885.952297,5623.422610
3,сотрудник,267628.550329,4124.747207
4,пенсионер,158616.077870,340266.072047


In [116]:
df1.isna().sum()

income_type         0
total_income     2174
days_employed    2174
dtype: int64

### Для проверки правильности заполнения NaN-ов посмотрим на текущие статистики по выборке, и на них же - после обработки NaN.

In [117]:
stats_before = df1.describe()

Применим к столбцам df1 операцию замены NaN на .median().

Описание этой операции см. в коде ниже.

In [118]:
df1.loc[:,('total_income')] = df1.groupby('income_type')['total_income'].apply(lambda x: x.fillna(x.median()))

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [119]:
df1.loc[:,('days_employed')] = df1.groupby('income_type')['days_employed'].apply(lambda x: x.fillna(x.median()))

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [120]:
df1.isna().sum()

income_type      0
total_income     0
days_employed    0
dtype: int64

### Видим, что NaN-ов действительно больше нет. Проверим изменения статистик.

In [121]:
stats_after = df1.describe()

In [123]:
print("Stats Before")
print(stats_before)
print("Stats After")
print(stats_after)

Stats Before
       total_income  days_employed
count  1.935100e+04   19351.000000
mean   1.674223e+05   66914.728907
std    1.029716e+05  139030.880527
min    2.066726e+04      24.141633
25%    1.030532e+05     927.009265
50%    1.450179e+05    2194.220567
75%    2.034351e+05    5537.882441
max    2.265604e+06  401755.400475
Stats After
       total_income  days_employed
count  2.152500e+04   21525.000000
mean   1.652253e+05   67299.486032
std    9.804367e+04  139401.804684
min    2.066726e+04      24.141633
25%    1.077982e+05    1025.549623
50%    1.425944e+05    1993.522017
75%    1.955499e+05    5347.024506
max    2.265604e+06  401755.400475


Видим, что статистики немного изменились - как мы и ожидали.

Поместим новые столбцы без NaN в исходный датасет:

In [124]:
data.total_income = df1.total_income

In [125]:
data.days_employed  = df1.days_employed 

In [126]:
data.isna().sum()

Unnamed: 0          0
children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Отбросим все знаки после запятой в наших столбцах - по логике задачи они не должны влиять на результат, а для компьютера так гораздо удобнее:

In [127]:

data.days_employed = data['days_employed'].astype(int)
data.total_income = data['total_income'].astype(int)

### Код, поясняющий заполнение Nan медианами.

Создадим "игрушечный" датасет: в качестве групп в нем будут значения в столбце name - всего две группы First и Second.

В качестве столбцов, по которым будем искать медиану и заполнять - col1 и col2 с числовыми значениями.

In [145]:
df = pd.DataFrame({'col1':[float('nan'), 1, 10, 5, 3, 4], 'col2':[1, 8, 6, float('nan'), 2, 0], 'name':['First', 'Second', 'First', 'Second', 'First', 'Second']})

In [146]:
df

,col1,col2,name
0,NaN,1.0,First
1,1.0,8.0,Second
2,10.0,6.0,First
3,5.0,NaN,Second
4,3.0,2.0,First
5,4.0,0.0,Second


Рассмотрим col1: медиана по группе First равна (10+3)/2 = 6.5, по группе Second - 4.

Рассмотрим col2: First: 2, Second: (0+8)/2 = 4.

Рассмотрим 3 кода:

In [150]:
df.groupby('name')['col1', 'col2'].median() # выводит выбранную статистику median по группе "name" для выбранных столбцов col1, col2

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


,col1,col2
name,,
First,6.5,2.0
Second,4.0,4.0


Предыдущее, оказывается, эквивалентно такой операции: группируем по name и как бы разделяем df на два: в одном только First, в другом - только Second. Далее применяем к столбцам этих "разделенных" датафреймов лямбда-функцию, которая, внимание! Заменяет СТОЛБЕЦ х на ЧИСЛО x.median(). Дальше полученное просто записывается в одну таблицу:

In [151]:
df.groupby('name')['col1', 'col2'].apply(lambda x: x.median()) 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


,col1,col2
name,,
First,6.5,2.0
Second,4.0,4.0


Ну и наконец, по аналогии с предыдущим кодом, заменим столбец x не на число, а на столбец, с заполненными nan (с помощью функции fillna()):

In [152]:
df.groupby('name')['col1', 'col2'].apply(lambda x: x.fillna(x.median()))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


col1  col2
name                
First  0   6.5   1.0
       2  10.0   6.0
       4   3.0   2.0
Second 1   1.0   8.0
       3   5.0   4.0
       5   4.0   0.0

# Конец пояснения

#Категоризация

Лемматизация

In [ ]:
from collections import Counter
Counter(data['education'])
data['education'] = data['education'].str.lower()
print(Counter(data['education']))

Counter(data['family_status'])
data['family_status'] = data['family_status'].str.lower()
print(Counter(data['family_status']))
print(Counter(data['income_type']))
print(Counter(data['purpose']))
print(Counter(data['gender']))


Counter({'среднее': 15233, 'высшее': 5260, 'неоконченное высшее': 744, 'начальное': 282, 'ученая степень': 6})
Counter({'женат / замужем': 12380, 'гражданский брак': 4177, 'не женат / не замужем': 2813, 'в разводе': 1195, 'вдовец / вдова': 960})
Counter({'сотрудник': 11119, 'компаньон': 5085, 'пенсионер': 3856, 'госслужащий': 1459, 'безработный': 2, 'предприниматель': 2, 'студент': 1, 'в декрете': 1})
Counter({'свадьба': 797, 'на проведение свадьбы': 777, 'сыграть свадьбу': 774, 'операции с недвижимостью': 676, 'покупка коммерческой недвижимости': 664, 'операции с жильем': 653, 'покупка жилья для сдачи': 653, 'операции с коммерческой недвижимостью': 651, 'покупка жилья': 647, 'жилье': 647, 'покупка жилья для семьи': 641, 'строительство собственной недвижимости': 635, 'недвижимость': 634, 'операции со своей недвижимостью': 630, 'строительство жилой недвижимости': 626, 'покупка недвижимости': 624, 'строительство недвижимости': 620, 'покупка своего жилья': 620, 'ремонт жилью': 612, 'покуп

In [ ]:
pip install pymorphy2

     |████████████████████████████████| 61kB 6.7MB/s 
     |████████████████████████████████| 8.2MB 28.4MB/s 


In [ ]:
pip install -U pymorphy2-dicts-ru

Requirement already up-to-date: pymorphy2-dicts-ru in /usr/local/lib/python3.7/dist-packages (2.4.417127.4579844)


In [ ]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [ ]:
morph.parse('своего')[0].normal_form

'свой'

In [ ]:
 data['purpose']

0                       покупка жилья
1             приобретение автомобиля
2                       покупка жилья
3          дополнительное образование
4                     сыграть свадьбу
                     ...             
21520               операции с жильем
21521            сделка с автомобилем
21522                    недвижимость
21523    на покупку своего автомобиля
21524           на покупку автомобиля
Name: purpose, Length: 21525, dtype: object

In [ ]:
lemmas = [[morph.parse(word)[0].normal_form for word in data['purpose']]]

In [ ]:
#lemmas

In [ ]:
data.head()

,Unnamed: 0,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


Обнаруженная **проблема**: к нормальной форме приводится только последнее слово предложения.

**Решение**: В процессе.


**Предложение:** чтобы категоризовать данные по "смыслу", заключенному в словах русского языка, неплохо было бы оставить во всех предложениях по одному слову, поставленному в нормальную форму.

Предполагается, что это будет существительное в единственном числе и именительном падеже. Пример:

"сыграть свадьбу" = "свадьба"

"на свадьбу" = "свадьба"


"на приобретение недвижемости" = "недвижемость" (как отсеять вариант "приобретение"????)


"на покупку автомобиля" = "автомобиль"

Тогда, во-первых будет существенно сокащен "словарь" данной задачи, а во-вторых станут доступны такие методы категоризации, как *One Hot Encoding*.

# Поиск зависимостей между признаками и целевой меткой:

In [ ]:
data[['children', 'total_income', 'debt']].corr()

,children,total_income,debt
children,1.000000,0.018485,0.018470
total_income,0.018485,1.000000,-0.012304
debt,0.018470,-0.012304,1.000000


**Проблема:** как интерпретировать? Значения близки к нулю.

**Решение 1**: избавиться от выбросов.

Отсортировать данные по столбцу total_income и "выкинуть" верхние и нижние 5% (10%).

Проблему с детьми это не решает. Возможно такой корреляции на самом деле нет.

Ссылка на тренажер по pandas и numpy:

https://stepik.org/lesson/190317/step/9


- весь 6 модуль курса очень полезен.